# Import packages

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Load Data

In [4]:
df = pd.read_csv('data/raw_data.csv', index_col='user_id')
df = df.iloc[:, 3:]
df

,Gims,Kaaris,Maes,Booba,Ninho,Guy2bezbar,Leto,Koba la D,Damso,Freeze Corleone,Vald,Nekfeu,Zola,JUL,Niska,Sofiane,Fresh,Lomepal,Hamza,Laylow
user_id,,,,,,,,,,,,,,,,,,,,
1,2,3,3,2,5,5,4,5,5,3,4,4,5,1,5,3,3,1,3,3
2,2,2,3,2,3,1,3,2,4,3,3,5,3,2,4,3,1,3,3,5
3,2,2,4,3,3,1,3,2,5,5,4,5,4,4,4,4,1,2,2,4
4,2,3,3,2,4,3,4,4,2,4,5,4,5,1,2,2,1,3,2,4
5,2,2,3,3,5,3,4,3,4,5,3,4,3,2,4,3,3,1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
136,1,1,2,1,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2
137,3,1,2,5,5,3,3,1,1,2,2,1,1,4,3,3,2,1,1,1


# Nearest user

In [5]:
df_distance_user = pd.DataFrame(
    squareform(pdist(df.values)),
    columns = df.index,
    index = df.index
)

In [6]:
def get_n_nearest_user(df_distance_user, id_user, n_user):
    distance_one_user = df_distance_user[id_user].sort_values().reset_index()[1:]
    distance_one_user.columns = ['user_id', 'distance']
    list_nearest_user = list(distance_one_user.iloc[:n_user]['user_id'])
    return list_nearest_user

In [7]:
get_n_nearest_user(df_distance_user, id_user=20, n_user=5)

[130, 5, 81, 63, 87]

In [8]:
df.loc[[20, 130, 5, 81, 63, 87]]

,Gims,Kaaris,Maes,Booba,Ninho,Guy2bezbar,Leto,Koba la D,Damso,Freeze Corleone,Vald,Nekfeu,Zola,JUL,Niska,Sofiane,Fresh,Lomepal,Hamza,Laylow
user_id,,,,,,,,,,,,,,,,,,,,
20,1,3,1,3,5,2,3,2,4,5,5,3,2,3,4,3,1,2,2,2
130,2,2,2,2,5,1,2,2,4,4,4,3,3,1,5,3,1,1,3,2
5,2,2,3,3,5,3,4,3,4,5,3,4,3,2,4,3,3,1,3,3
81,2,2,2,4,2,1,1,1,4,4,4,3,1,4,4,3,1,2,3,2
63,1,3,4,4,5,2,3,2,5,5,5,5,3,5,4,1,1,1,3,2
87,1,2,4,1,5,1,3,3,4,3,5,4,3,2,4,3,1,3,3,1


# Reco

In [10]:
list_all_favorite_artist = list(df[df >= 4].apply(lambda x: list(x.dropna().index), axis=1).values)
df['favorite_artist'] = [", ".join(i) for i in list_all_favorite_artist]

In [12]:
correlation_matrix = df.corr()

C:\Users\flore\AppData\Local\Temp\ipykernel_24756\3450543936.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [13]:
def get_reco_one_rappeur(correlation_matrix, rappeur, n_rappeur):
    return list(correlation_matrix[rappeur].sort_values(ascending=False)[1:n_rappeur+1].index)

In [14]:
# 70 chaque artiste et 77 tout ce qui est mainstream

In [15]:
favorite_artist = list_all_favorite_artist[0]
favorite_artist

['Ninho',
 'Guy2bezbar',
 'Leto',
 'Koba la D',
 'Damso',
 'Vald',
 'Nekfeu',
 'Zola',
 'Niska']

In [16]:
if len(favorite_artist) > 0:
    rappeur_reco = []
    for rappeur in favorite_artist:
        rappeur_reco.extend(get_reco_one_rappeur(correlation_matrix, rappeur, 2))

    rappeur_reco = list(set(rappeur_reco))
    rappeur_reco = [rappeur for rappeur in rappeur_reco if rappeur not in favorite_artist]
    rappeur_reco = 'Artistes recommandés : ' + ', '.join(rappeur_reco)
else:
    rappeur_reco = "Pas d'artiste à recommander"

rappeur_reco

'Artistes recommandés : Freeze Corleone, Maes, Lomepal, Laylow'

In [17]:
def get_reco_rappeur_all_favorite(favorite_artist, correlation_matrix):
    if len(favorite_artist) > 0:
        rappeur_reco = []
        for rappeur in favorite_artist:
            rappeur_reco.extend(get_reco_one_rappeur(correlation_matrix, rappeur, 2))

        rappeur_reco = list(set(rappeur_reco))
        rappeur_reco = [rappeur for rappeur in rappeur_reco if rappeur not in favorite_artist]
        rappeur_reco = ", ".join(rappeur_reco)
    else:
        rappeur_reco = "Pas d'artiste à recommander"

    return rappeur_reco

In [18]:
reco = []
for favorite_artist in list_all_favorite_artist:
    reco.append(get_reco_rappeur_all_favorite(favorite_artist, correlation_matrix))

In [19]:
reco

['Freeze Corleone, Maes, Lomepal, Laylow',
 'Zola, Vald, Hamza, Maes, Lomepal, Ninho',
 'Leto, Kaaris, Hamza, Ninho, Lomepal',
 'Hamza, Damso, Maes, Lomepal, Niska',
 'Vald, Maes, Laylow, Lomepal, Zola',
 'Kaaris, Leto, Hamza, Niska',
 "Pas d'artiste à recommander",
 'Hamza, Maes, Lomepal, Zola, Niska',
 'Kaaris, Zola, Vald, Lomepal, Laylow',
 "Pas d'artiste à recommander",
 "Pas d'artiste à recommander",
 'Freeze Corleone, Laylow',
 "Pas d'artiste à recommander",
 'Kaaris, Zola, Vald, Niska',
 'Freeze Corleone, Nekfeu, Damso, Niska',
 'Maes, Zola, Niska',
 'Vald',
 'Kaaris, Vald, Maes, Nekfeu, Niska',
 'Kaaris, Leto, Freeze Corleone, Laylow, Damso',
 'Maes, Nekfeu, Laylow',
 'Kaaris, Zola, Hamza, Niska',
 'Leto, Kaaris, JUL, Guy2bezbar',
 'Leto, Freeze Corleone, Hamza',
 "Pas d'artiste à recommander",
 'Nekfeu, Damso',
 "Pas d'artiste à recommander",
 'Freeze Corleone, Maes, Nekfeu, Zola, Niska',
 "Pas d'artiste à recommander",
 'Damso',
 'Freeze Corleone, Damso',
 'Kaaris, Vald, Lome

In [20]:
df['reco'] = reco

In [25]:
df[['favorite_artist', 'reco']]

,favorite_artist,reco
user_id,,
1,"Ninho, Guy2bezbar, Leto, Koba la D, Damso, Val...","Freeze Corleone, Maes, Lomepal, Laylow"
2,"Damso, Nekfeu, Niska, Laylow","Zola, Vald, Hamza, Maes, Lomepal, Ninho"
3,"Maes, Damso, Freeze Corleone, Vald, Nekfeu, Zo...","Leto, Kaaris, Hamza, Ninho, Lomepal"
4,"Ninho, Leto, Koba la D, Freeze Corleone, Vald,...","Hamza, Damso, Maes, Lomepal, Niska"
5,"Ninho, Leto, Damso, Freeze Corleone, Nekfeu, N...","Vald, Maes, Laylow, Lomepal, Zola"
...,...,...
135,,Pas d'artiste à recommander
136,,Pas d'artiste à recommander
137,"Booba, Ninho, JUL","Kaaris, Maes, Niska"
